# Cryptographically securing control messages

Control message hijacking and faking is a major concern in the protection of ICS. To resolve this problem two cryptographic solutions have been suggested. 

The first and most direct being encyrption/decryption and the other being comparing message hashes with an added secret. 

## Encryption and Decryption 

We shall be using the open source fernet library that is part of the cryptography package. 

In [13]:
from cryptography.fernet import Fernet #Importing the library 

key = Fernet.generate_key() # A key is generated that will be used for both encryption and decryption. Typically fernet 
                            # creates keys with higher entropy so we are using that here. This key shall be secure between
                            # the PLC and the controller.

In [14]:
fernet = Fernet(key) # Here we initialize the fernet class with the generated key

In [15]:
plc_message = input("Enter the control message to be sent: ") # For demonstration only.

encrypted_plc_message = fernet.encrypt(plc_message.encode()) # Encryption

print("Original message: ", plc_message) #Original message for demonstration 

print("Encrypted message: ", encrypted_plc_message) #Encrypted message for demonstration 

decrypted_plc_message = fernet.decrypt(encrypted_plc_message).decode() #We decrypt the encrypted message from the PLC
 
print("Decrypted message: ", decrypted_plc_message) #Printing the decrypted message for demonstration

Enter the control message to be sent: TEMP = 500K
Original message:  TEMP = 500K
Encrypted message:  b'gAAAAABiskKGDYpVHsPU8eeT6XeLW11u90wyrQlprV5yBTdsaciQ9hIV16t5j9k2zpanwV9WHbmK5wX-ECQvzEDGz65J5y-sKQ=='
Decrypted message:  TEMP = 500K


This method is relatively secure since anyone trying to create a fernet instance without knowing the key would be unable to decrypt the control messages or send their own encrypted messages. Moreover, a unique key can be created for every message

## Hashing

We shall be using the hashlib library to compare the hashes of the sent messages with the received ones. 

In [20]:
import hashlib #Importing the hashing library

def hash_message(message):
    sha_signature = hashlib.sha512(message.encode()).hexdigest() # Converts the string to a 512 hash and presents it in a readable form                                                                # a readable manner using hexdigest
    return sha_signature


In [21]:
import secrets
secret = secrets.token_urlsafe(16) # Generating secret for secure communication of the hash. Only known to PLC and controller

In [22]:
control_message = input() # Message for demonstration. 
secure_control_message = control_message + secret
hashed_control_message = hash_message(secure_control_message) #Hashes the given control message with added secret

print("Secure Control message: " , secure_control_message) #For demonstration
print("Hashed control message: " ,hashed_control_message) #For demonstartion

transmitted_control_message = { 'control_message' : control_message, 'hash':hashed_control_message}
print(transmitted_control_message)

TEMP = 200K
Secure Control message:  TEMP = 200KZU8eC6XSFQpwmaI3ne7WVQ
Hashed control message:  5dbe3ed03bdef42e1d414943175839c5ca8cec8663c543b4a096f0e6cc93e24793f2472cbd695064d423662bb7e73a1a37c959aca8b8d31575c411091815c445
{'control_message': 'TEMP = 200K', 'hash': '5dbe3ed03bdef42e1d414943175839c5ca8cec8663c543b4a096f0e6cc93e24793f2472cbd695064d423662bb7e73a1a37c959aca8b8d31575c411091815c445'}


In [23]:
## On the controller end after transmission through secured channel

received_message = transmitted_control_message['control_message'] # Extracts control message from the transmission
secure_received_message = received_message + secret # Adds the secret to the received message
 
hashed_received_message = hash_message(secure_received_message) #Hashes the received message on the controller side

if(hashed_received_message == transmitted_control_message['hash'] ):                      #If the hash of the transmitted message + secret match the hash
    print(f"Received message: {received_message}, Message transmitted and authenticated") # of received message + secret we know that the message has been authenticated
else:                                                                   
    print("Red Flag. Data modified. Cyberattack alert") 




Received message: TEMP = 200K, Message transmitted and authenticated


### End of File